In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [4]:
# shared-layer

shared_embedding = layers.Embedding(10000, 64)

In [5]:
# funtional-api 

text_input_a = keras.Input(shape=(None,),dtype='int32')
text_input_b = keras.Input(shape=(None,),dtype='int32')


# 透過funtional api shared-layer

encoded_input_a = shared_embedding(text_input_a)
emcoded_input_b = shared_embedding(text_input_b)

In [6]:
# 特徵獲取, 透過functional api

# 預訓練 model
vgg19 = keras.applications.VGG19()

# 獲取每一個layer的output
features_list = [layer.output for layer in vgg19.layers]

# 透過上面去取得特徵萃取
feat_extraction_model = keras.Model(inputs=vgg19.input, outputs=features_list)

574717952/574710816 [==============================] - 6s 0us/step


In [10]:
features_list

[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 224, 224, 64) dtype=float32 (created by layer 'block1_conv1')>,
 <KerasTensor: shape=(None, 224, 224, 64) dtype=float32 (created by layer 'block1_conv2')>,
 <KerasTensor: shape=(None, 112, 112, 64) dtype=float32 (created by layer 'block1_pool')>,
 <KerasTensor: shape=(None, 112, 112, 128) dtype=float32 (created by layer 'block2_conv1')>,
 <KerasTensor: shape=(None, 112, 112, 128) dtype=float32 (created by layer 'block2_conv2')>,
 <KerasTensor: shape=(None, 56, 56, 128) dtype=float32 (created by layer 'block2_pool')>,
 <KerasTensor: shape=(None, 56, 56, 256) dtype=float32 (created by layer 'block3_conv1')>,
 <KerasTensor: shape=(None, 56, 56, 256) dtype=float32 (created by layer 'block3_conv2')>,
 <KerasTensor: shape=(None, 56, 56, 256) dtype=float32 (created by layer 'block3_conv3')>,
 <KerasTensor: shape=(None, 56, 56, 256) dtype=float32 (created by layer 'block3_conv4')>,

In [12]:
img = np.random.random((1, 224, 224, 3)).astype('float32')

extracted_features = feat_extraction_model(img)
for f in extracted_features:
    print(f.shape)
extracted_features

(1, 224, 224, 3)
(1, 224, 224, 64)
(1, 224, 224, 64)
(1, 112, 112, 64)
(1, 112, 112, 128)
(1, 112, 112, 128)
(1, 56, 56, 128)
(1, 56, 56, 256)
(1, 56, 56, 256)
(1, 56, 56, 256)
(1, 56, 56, 256)
(1, 28, 28, 256)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 14, 14, 512)
(1, 14, 14, 512)
(1, 14, 14, 512)
(1, 14, 14, 512)
(1, 14, 14, 512)
(1, 7, 7, 512)
(1, 25088)
(1, 4096)
(1, 4096)
(1, 1000)


[<tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
 array([[[[1.1615647e-01, 9.3847501e-01, 8.4075302e-02],
          [7.8132385e-01, 2.7763164e-01, 4.5831528e-01],
          [4.0659279e-01, 5.8287686e-01, 9.9263018e-01],
          ...,
          [6.6240644e-01, 9.3373805e-01, 3.3714491e-01],
          [7.1063653e-02, 5.4239437e-02, 3.1005710e-01],
          [4.2656876e-02, 7.7623266e-01, 2.7649075e-01]],
 
         [[1.2999147e-01, 3.4344272e-04, 8.5561186e-01],
          [7.1530151e-01, 9.0937042e-01, 6.6125399e-01],
          [7.8257024e-01, 5.3544259e-01, 5.5314279e-01],
          ...,
          [7.1250796e-01, 4.1218284e-01, 4.4498032e-01],
          [3.4575668e-01, 2.0369235e-01, 8.8786870e-01],
          [6.5682667e-01, 4.3226820e-01, 4.9224105e-01]],
 
         [[3.1547534e-01, 7.8058600e-01, 9.6935910e-01],
          [7.0176786e-01, 2.1253718e-01, 8.0854529e-01],
          [4.2799684e-01, 2.3347034e-01, 4.2000335e-01],
          ...,
          [6.8202400e-01, 7.4116373

In [25]:
## 客製化Layer 去延伸

class CustomDense(layers.Layer):
    def __init__(self, units=32):
        super(CustomDense, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer='random_normal',
            trainable=True
        )
    def call(self, x):
        return tf.matmul(x, self.w) + self.b
    
    def get_config(self):
        """  用來init 搭配Class method .from_config """
        return {
            'units': self.units
        }
    
    # default implementation
    # def from_config(cls, config):
    #     return cls(**config)

inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)
config = model.get_config()

new_model = keras.Model.from_config(config=config, custom_objects={'CustomDense': CustomDense})